# **2021 Fall AI701 Homework assignment 3**
Read the following problem sheet and submit your completed $\texttt{ipynb}$ file.

## Problem 1 (70 pts)

We will going to implement the sequential importance sampling (SIS) and sequential importance resampling (SIR) for a simple state-space model. The state-space model is defined as follows:

$\displaystyle x_1 \sim \mathcal{N}\bigg(\mu, \frac{\sigma^2}{1 - \rho^2}\bigg).$

$\displaystyle x_t|x_{t-1} \sim \mathcal{N}(\mu + \rho(x_{t-1} - \mu), \sigma^2).$

$\displaystyle y_t|x_t \sim \mathcal{N}(0, e^{x_t}).$

The following codes define the functions required for generating sequences from this model.

In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.special import logsumexp

In [ ]:
# parameters, we will use a fixed set of parameters throughout this homework.
params = {'mu':-1.0, 'rho':0.95, 'sigma':0.2}

In [ ]:
# sample x_1 ~ p(x_1).
def sample_x1(num_samples=1):
    std = params['sigma'] / np.sqrt(1 - params['rho']**2)    
    return params['mu'] + std*npr.normal(size=num_samples)
        
# sample x_t ~ p(x_t|x_{t-1}) with x_prev denoting x_{t-1}.
def sample_x(x_prev):
    mean = params['mu'] + params['rho']*(x_prev - params['mu'])
    shape = None if type(x_prev)==float else x_prev.shape
    return mean + params['sigma']*npr.normal(size=shape)

# sample y_t ~ p(y_t|x_t).
def sample_y(x):    
    shape = None if type(x)==float else x.shape
    return np.exp(0.5*x)*npr.normal(size=shape)

Using these functions, the following code actually generates a sequence $\{y_t\}_{t=1}^T$.

In [ ]:
T = 100
x_true = np.zeros(T)
y = np.zeros(T)
npr.seed(42)

x_true[0] = sample_x1()
y[0] = sample_y(x_true[0])
for t in range(1, T):
    x_true[t] = sample_x(x_true[t-1])
    y[t] = sample_y(x_true[t])

fig, axes = plt.subplots(1, 2, figsize=(9, 4))
axes[0].set_title(r'$x_{true}$')
axes[0].plot(range(T), x_true)
axes[1].set_title(r'$y$')
axes[1].plot(range(T), y)

To define sequential importance sampling algorithm, we choose the simplest possible proposal distribution; the prior transition distribution.
$
q(x_t | x_{1:t-1}, y_{1:t}) = p(x_t | x_{t-1}).
$

In [ ]:
# propose a sample x_t ~ q(x_t|x_{1:t-1}, y_{1:t}) = p(x_t|x_{t-1})
def propose_x(x_prev):
    return sample_x(x_prev)

# computes the log-likelihood p(y_t|x_t)
def log_likel(y, x):
    var = np.exp(x)
    ll = -0.5*y**2/var - 0.5*np.log(2*np.pi*var)
    return ll

Now we are ready to write the sequential importance sampling algorithm. Let's first start by writing the function to initialize set of particles at $t=1$.

In [ ]:
# initialize num_particles number of particles.
# particles are represented with a dictionary:
# 'x' denotes the actual samples of the states with shape (num_particles) * (num_timesteps).
# 'log_w' denotes log of unnormalized importance weights with shape (num_particles) * (num_timesteps).
# After initialization, particles['x'] and particles['log_w'] should have shape (num_particles) * 1.
def init_particles(y, num_particles):
    x = sample_x1(num_samples=num_particles)
    log_w = log_likel(y, x)
    particles = {'num_particles':num_particles, 
                 'x':x[:,None], 
                 'log_w':log_w[:,None]}
    return particles

(a) (20pts) Complete the following code to extend the set of particles for a new observation $y_t$. Taking the set of particles constructed up to time $t-1$ and $y_t$, the function should output update particles with states and unnomarlized importance weights constructed up to time $t$.

In [ ]:
# particles['x'] and particles['log_w'] have shape (num_particles) * (t-1)
# given y_t, the following function should return the update set of particles
# where both particles['x'] and particles['log_w'] have shape (num_particles) * t
def SIS_extend_particles(y, particles):
    # complete this part
    ######################################################################
 
    ######################################################################
    return particles

# Once you have the function to extend particles, the SIS algorithm is straightforward as follows.
def run_SIS(y, num_particles):
    particles = init_particles(y[0], num_particles)
    for t in range(1, len(y)):
        particles = SIS_extend_particles(y[t], particles)
    return particles

(b) (10pts) Complete the following functions computing the estimate of the log marginal $\log p(y_{1:T})$ and the effective sample size using the particles constructed up to time $T$. 

Hint: use $\texttt{logsumexp}$ function for numerical stability.

In [ ]:
# return a scalar value computing the estimate of the log marginal likelihood log p(y_{1:T})
def SIS_log_marginal(particles):
    # complete this part
    ######################################################################

    ######################################################################

# return a scalar value computing the effective sample size 
def compute_ess(particles):
    # complete this part
    ######################################################################
  
    ######################################################################

(c) (10pts) Since we have the true states $x_{\text{true}}$ that we have used for the data generation, we can compute the estimation error for the estimatated states. That is, computing the expected error defined as

$
\displaystyle \mathcal{E}(x_\text{true}) = \frac{1}{T} \mathbb{E}_{p(x_{1:T}|y_{1:T})}\bigg[ \sum_{t=1}^T (x_t - x_{\text{true}, t})^2\bigg] = \frac{1}{T} \int \sum_{t=1}^T (x_t - x_{\text{true}, t})^2 p(x_{1:T}|y_{1:T}) \mathrm{d} x_{1:T}.
$

Complete the following code computing the approximation of this expected error using the particles and importance weights we construct with SIS. Then run the complete codes to get the results.

In [ ]:
# return a scalar estimate of the expected error
def estimate_error(x_true, particles):
    # complete this part
    ######################################################################
          
    ######################################################################

num_particles = 200
particles = run_SIS(y, num_particles)

lm = SIS_log_marginal(particles)
ess = compute_ess(particles)
err = estimate_error(x_true, particles)
print(f'SIS log marginal {lm:.4f}, ESS {ess:.4f}, expected error {err:.4f}')

(d) (5pts) For visualization, we will compute the mean and variances of the estimated states.

$
\displaystyle 
\mathbb{E}_{p(x_{1:T}|y_{1:T})}[x_t] = \int x_t p(x_{1:T}|y_{1:T}) \mathrm{d}x_{1:T}, \quad
\mathrm{Var}_{p(x_{1:T}|y_{1:T})}[x_t] = \int (x_t - \mathbb{E}_{p(x_{1:T}|y_{1:T})})^2 p(x_{1:T}|y_{1:T}) \mathrm{d}x_{1:T}.
$

Complete the following codes computing the approximation of expected means and variances using the particles and importance weights we get from SIS. Run the visualization code to see if the algorithm works well.

In [ ]:
# return a pair of T-dim vectors estimating the mean and variance of the sequence x_{1:T}
def estimate_x_mean_and_var(particles):
    # complete this part
    ######################################################################

    ######################################################################
    return x_mean, x_var

plt.figure(figsize=(5,5))
plt.title('SIS estimated states')
x_mean, x_var = estimate_x_mean_and_var(particles)
x_std = np.sqrt(x_var)
plt.fill_between(range(T), x_mean-x_std, x_mean+x_std, 
                 facecolor='b', edgecolor=None, alpha=0.2)
plt.plot(range(T), x_mean, color='b', label=r'$x_{mean}$')
plt.plot(range(T), x_true, '--r', label=r'$x_{true}$', )
plt.legend(fontsize=15)

(e) (25pts) Complete the following functions required for sequential importance resampling. You may use $\texttt{npr.choice}$ function for categorical resampling step. Run the algorithm to verify your implementation.

In [ ]:
# particles['x'] and particles['log_w'] have shape (num_particles) * (t-1)
# given y_t, the following function should return the update set of particles
# where both particles['x'] and particles['log_w'] have shape (num_particles) * t
def SIR_extend_particles(y, particles):
    # complete this part
    ######################################################################
             
    ######################################################################
    return particles

def run_SIR(y, num_particles):
    particles = init_particles(y[0], num_particles)
    for t in range(1, len(y)):
        particles = SIR_extend_particles(y[t], particles)
    return particles

# return a scalar value computing the estimate of the log marginal likelihood log p(y_{1:T})
def SIR_log_marginal(particles):
    # complete this part
    ######################################################################
    
    ######################################################################

num_particles = 200
particles = run_SIR(y, num_particles)

lm = SIR_log_marginal(particles)
ess = compute_ess(particles)
err = estimate_error(x_true, particles)
print(f'SIR log marginal {lm:.4f}, ESS {ess:.4f}, expected error {err:.4f}')

plt.figure(figsize=(5,5))
plt.title('SIR estimated states')
x_mean, x_var = estimate_x_mean_and_var(particles)
x_std = np.sqrt(x_var)
plt.fill_between(range(T), x_mean-x_std, x_mean+x_std, 
                 facecolor='b', edgecolor=None, alpha=0.2)
plt.plot(range(T), x_mean, color='b', label=r'$x_{mean}$')
plt.plot(range(T), x_true, '--r', label=r'$x_{true}$', )
plt.legend(fontsize=15)

## Problem 2 (30pts)
Consider sampling from the following target distribution,

$$
p(x) \propto \exp( - U(x)), \quad U(x) = \frac{1}{2} x^\top A x, \quad A = \begin{bmatrix} 2 & 1.2 \\ 1.2 & 2 \end{bmatrix}, 
$$

where $x \in \mathbb{R}^2$ and $A \in \mathbb{R}^{2\times 2}$. We can visualize the contour plot of the density function as follows:


In [ ]:
A = np.array([[2, 1.2], [1.2, 2]])
# x: 2 dimensional vector or N times 2 dimensional matrix.
def energy_function(x):    
    if x.ndim == 1:
        return 0.5 * np.inner(x, np.dot(A, x)) 
    elif x.ndim == 2:
        return 0.5 * (x * np.dot(x, A)).sum(-1)
    else:
        raise NotImplementedError

def plot_density(alpha=1.0):
    nx, ny = 50, 50
    x = np.linspace(-3, 3, nx)
    y = np.linspace(-3, 3, ny)
    xx, yy = np.meshgrid(x, y)
    z = np.exp(-energy_function(np.concatenate([xx.reshape((-1, 1)), yy.reshape((-1, 1))], -1)))
    plt.contour(x, y, z.reshape((nx, ny)), cmap='inferno', alpha=alpha)

plot_density()

We will sample from this target distribution using Hamiltonian Monte Carlo. The kinetic energy function and Hamiltonian function are assumed to be

$
K(p) = \frac{1}{2}p^\top p, \quad H(x, p) = U(x) + K(p).
$

Complete the following codes running HMC.

In [ ]:
# computes grad_x U(x) for 2 dimensional vector x.
def energy_function_grad(x):
    # complete this part
    ######################################################################
    
    ######################################################################

# compute the kinetic energy for 2 dimensional momentum vector p.
def kinetic_energy_function(p):
    return 0.5 * np.inner(p, p)

# run HMC and collect (num_samples) number of samples.
# x0: initial position, 2 dimensional vector.
# eps: step size
# L: number of leapfrog integrations
# output: (num_samples) * 2 dimensional matrix collecting the samples.
def run_HMC(x0, eps, L, num_samples):
    # complete this part
    ######################################################################
   
    ######################################################################

Run the following code the verify your sampler.

In [ ]:
x0 = npr.rand(2)
eps = 0.01
L = 30
num_samples = 5000
samples = run_HMC(x0, eps, L, num_samples)
plot_density()
plt.scatter(samples[:,0], samples[:,1], alpha=0.5, s=20, color='k')